In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [3]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-HR/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 0.8988719 ,  0.96348953,  1.0283111 , ...,  0.8222215 ,
          0.830732  ,  0.8506962 ],
        [ 0.8791747 ,  0.92337066,  0.9691958 , ...,  0.86992383,
          0.85690415,  0.8519915 ],
        [ 0.9184422 ,  0.93199897,  0.942068  , ...,  0.9151408 ,
          0.8993026 ,  0.90063745],
        ...,
        [ 0.13769086,  0.13756882,  0.13742055, ...,  0.13777447,
          0.13778755,  0.13776678],
        [ 0.08825146,  0.08818546,  0.08807883, ...,  0.08840771,
          0.08838986,  0.08833221],
        [ 0.04274025,  0.04271388,  0.04268492, ...,  0.04276316,
          0.04276475,  0.04275429]],

       [[ 0.5790393 ,  0.6617941 ,  0.74485517, ...,  0.31355408,
          0.41584972,  0.50048196],
        [ 0.5240259 ,  0.6209582 ,  0.7101083 , ...,  0.26176855,
          0.34555072,  0.42697474],
        [ 0.470944  ,  0.58239883,  0.667455  , ...,  0.1728884 ,
          0.24446975,  0.34353346],
        ...,
        [ 0.05481615,  0.05600079,  0.05719237, ...,  

In [4]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [5]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [6]:
AO_ds.to_netcdf('MPI-ESM1-2-HR-AOindex-NDJF-Daily-1980-2014.nc')

In [7]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.642155, dtype=float32)
Coordinates:
    time     datetime64[ns] 1988-12-17T12:00:00

In [8]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [9]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.9212923, dtype=float32)
Coordinates:
    time     datetime64[ns] 2000-02-02T12:00:00

In [10]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [11]:
zLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-HR/processed/NDJF_New_zg_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_18500101-20091231.nc')

In [12]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [13]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 192, lon: 384)>
[354631680 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat      (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [14]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [15]:
#z145E40N

In [16]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [17]:
#z20E55N

In [18]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [19]:
#z75E55N

In [20]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [21]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [22]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [23]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([ 1.16463766,  0.98076392,  0.0730339 , ..., -1.15338277,
       -0.97989992, -0.68891561])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 145.3

In [24]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [25]:
EU_ds.to_netcdf('MPI-ESM1-2-HR-EUindex-NDJF-Daily-1980-2014.nc')